# Particle database

This notebook explains the difference between the particle list in `pycompwa`
and the list used in the ComPWA C++ back-end. The two are only related the
way they import and export XML files.

## pycompwa particle list

In [ ]:
from pycompwa.expertsystem.ui.system_control import StateTransitionManager
from pycompwa.expertsystem.state.particle import particle_list

initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma"), ("pi0"), ("pi0")]
tbd_manager = StateTransitionManager(
    initial_state, final_state,
    formalism_type='helicity',
    topology_building='isobar')
original_number_of_entries = len(particle_list)
print('loaded', original_number_of_entries, 'particles')

### (1) through Python

In [ ]:
particle_list['D*(2010)+']

Although this is a bit arduous to work with, it is possible to directly
manipulate the entries in there through Python. Whatever modifications to
the ``particle_list`` instance are later on also used in the expert system.
In this case, for instance, we can set a wider width for the $D^*$:

In [ ]:
particle_list['D*(2010)+']['DecayInfo']['Parameter'][0]['Value'] = 0.01

In [ ]:
from pycompwa.expertsystem.state import particle

In [ ]:
energies = [4180, 4220, 4420, 4600]  # MeV

for counter, energy in enumerate(energies, 1):
    new_particle = particle.get_particle_copy_by_name("EpEm")
    new_particle["Parameter"]["Value"] = energy / 1e3  # GeV
    pid = int(new_particle['Pid']) + counter
    new_particle["@Name"] += str(energy)  # new name to not overwrite the old
    new_particle["Pid"] = str(pid) # set unique PID
    particle.add_to_particle_list(new_particle)

assert original_number_of_entries + len(energies) == len(particle_list)

In [ ]:
from IPython.display import display
display(particle.particle_list["EpEm"]["Parameter"])
display(particle.particle_list["EpEm4180"]["Parameter"])

In [ ]:
particle.particle_list['D*(2010)+']['DecayInfo']['Parameter'][0]['Value'] = 0.01

Finally, you can write particle to another XML file, so that you can have
this particle list manipulation in a separate Python script:

In [ ]:
particle.write_particle_list_to_xml('new_particle_list.xml')

### (2) through XML

Alternatively, you can copy the default XML file and manipulate it by hand
(copy a section, tweak some names, change some parameters) and load it as
follows:

In [ ]:
particle.load_particle_list_from_xml('../particle_list.xml')

Note that duplicate values in the particle list are overwritten, but that old
values are kept:

In [ ]:
print('Number of entries in particle_list')
print('  old:', original_number_of_entries)
print('  new:', len(particle.particle_list))

In [ ]:
particle_list['D*(2010)+']['DecayInfo']['Parameter'][0]['Value']

In [ ]:
particle_list['EpEm4420']['Parameter']

## ComPWA particle list

In [ ]:
import pycompwa.ui

particles = pycompwa.ui.read_particles('../particle_list.xml')

You can append particles to this instance from another list with

In [ ]:
pycompwa.ui.Logging('error')  # switch off overwrite warnings
pycompwa.ui.insert_particles(particles, '../particle_list.xml')